References and Borrowing
---

What is Ownership?
---
***

Ownership is a set of rules that govern how a Rust program manages memory.  
All programs have to manage the way they use a computer’s memory while running.  
Some languages have garbage collection that regularly looks for no-longer-used  
memory as the program runs; in other languages, the programmer must explicitly  
allocate and free the memory. Rust uses a third approach: memory is managed  
through a system of ownership with a set of rules that the compiler checks.  
If any of the rules are violated, the program won’t compile. None of the  
features of ownership will slow down your program while it’s running.

Because ownership is a new concept for many programmers, it does take some time  
to get used to. The good news is that the more experienced you become with Rust  
and the rules of the ownership system, the easier you’ll find it to naturally  
develop code that is safe and efficient. Keep at it!

When you understand ownership, you’ll have a solid foundation for understanding  
the features that make Rust unique. In this chapter, you’ll learn ownership by  
working through some examples that focus on a very common data structure:  

- strings.

### Stack and Heap
***

Many programming languages don’t require you to think about the stack and the  
heap very often. But in a systems programming language like Rust, whether a  
value is on the stack or the heap affects how the language behaves and why you  
have to make certain decisions. Parts of ownership will be described in relation  
to the stack and the heap later in this chapter, so here is a brief explanation  
in preparation.

Both the stack and the heap are parts of memory available to your code to use at  
runtime, but they are structured in different ways. The stack stores values in  
the order it gets them and removes the values in the opposite order. This is  
referred to as last in, first out. Think of a stack of plates: when you add more  
plates, you put them on top of the pile, and when you need a plate, you take one  
off the top. Adding or removing plates from the middle or bottom wouldn’t work  
as well! Adding data is called pushing onto the stack, and removing data is  
called popping off the stack. All **`data stored on the stack`** must have a known,  
**`fixed size`**. Data with an **`unknown size at compile time`** or a size that might  
change must be **`stored on the heap`** instead.

The heap is less organized: when you put data on the heap, you request a certain  
amount of space. The memory allocator finds an empty spot in the heap that is  
big enough, marks it as being in use, and returns a pointer, which is the  
address of that location. This process is called allocating on the heap and is  
sometimes abbreviated as just allocating (pushing values onto the stack is not  
 considered allocating). Because the **`pointer to the heap is a known, fixed size,`**  
 you can **`store the pointer on the stack`**, but when you want the actual data, you  
 must follow the pointer. Think of being seated at a restaurant. When you enter,  
 you state the number of people in your group, and the host finds an empty table  
 that fits everyone and leads you there. If someone in your group comes late,  
 they can ask where you’ve been seated to find you.

Pushing to the stack is faster than allocating on the heap because the allocator  
never has to search for a place to store new data; that location is always at  
the top of the stack. Comparatively, allocating space on the heap requires more    
work because the allocator must first find a big enough space to hold the data  
and then perform bookkeeping to prepare for the next allocation.

Accessing data in the heap is slower than accessing data on the stack because  
you have to follow a pointer to get there. Contemporary processors are faster  
if they jump around less in memory. Continuing the analogy, consider a server at  
a restaurant taking orders from many tables. It’s most efficient to get all the  
orders at one table before moving on to the next table. Taking an order from  
table A, then an order from table B, then one from A again, and then one from B  
again would be a much slower process. By the same token, a processor can do its  
job better if it works on data that’s close to other data (as it is on the stack)  
rather than farther away (as it can be on the heap).

When your code calls a function, the values passed into the function (including,  
potentially, pointers to data on the heap) and the function’s local variables  
get pushed onto the stack. When the function is over, those values get popped  
off the stack.

Keeping track of what parts of code are using what data on the heap, minimizing  
the amount of duplicate data on the heap, and cleaning up unused data on the  
heap so you don’t run out of space are all problems that ownership addresses.  
Once you understand ownership, you won’t need to think about the stack and the  
heap very often, but knowing that the main purpose of ownership is to manage  
heap data can help explain why it works the way it does.


### Ownership Rules
***

First, let’s take a look at the ownership rules. Keep these rules in mind as we  
work through the examples that illustrate them:

- Each value in Rust has an owner.
- There can only be one owner at a time.
- When the owner goes out of scope, the value will be dropped.

### Variable Scope

In [26]:
{                       // s is not valid here, it’s not yet declared
    let s = "Hello";    // s is valid from this point forward
    // .. do stuff with s ..
}                       // this scope is now over, and s is dropped and no longer valid

In other words, there are two important points in time here:

- When s comes into scope, it is valid.
- It remains valid until it goes out of scope.

At this point, the relationship between scopes and when variables are valid is  
similar to that in other programming languages. Now we’ll build on top of this  
understanding by introducing the String type.

### The String Type

In [27]:
{
    let s = "Hello";               // string LITERAL
    println!("[LITERAL] {}", s);
    let mut s = String::from("Hello"); // string TYPE (shadow)
    println!("[TYPE] {}", s);
    s.push_str(", world!");        // push_str() appends a literal to a String
    println!("[TYPE is MUTABLE] {}", s);
}

[LITERAL] Hello
[TYPE] Hello
[TYPE is MUTABLE] Hello, world!


()

So, what’s the difference here? Why can String be mutated but literals cannot?  
The difference is in how these two types deal with memory.

### Memory and Allocation
***

In the case of a string literal, we know the `contents at compile time`, so the  
text `is hardcoded directly into the final executable`. This is why string  
literals are fast and efficient. But these properties only come from the string  
literal’s `immutability`. Unfortunately, we can’t put a blob of memory into the  
binary for each piece of text whose size is unknown at compile time and whose  
size might change while running the program.

With the `String type`, in order to `support a mutable, growable piece of text`, we  
need to `allocate an amount of memory on the heap`, `unknown at compile time`, to  
hold the contents. This means:

- The memory must be requested from the memory allocator at runtime.  

We need a way of returning this memory to the allocator when we’re done with our  
String.

That first part is done by us: when we call String::from, its implementation  
requests the memory it needs. This is pretty much universal in programming  
languages.

However, the second part is different. In languages with a garbage collector  
(GC), the GC keeps track of and cleans up memory that isn’t being used anymore,  
and we don’t need to think about it. In most languages `without a GC`, it’s our  
`responsibility to identify when memory is no longer being used` and to call code  
to `explicitly free it`, just as we did to request it. Doing this correctly has  
historically been a difficult programming problem. If we forget, we’ll waste  
memory. If we do it too early, we’ll have an invalid variable. If we do it twice,  
that’s a bug too. We need to pair exactly one allocate with exactly one free.

Rust takes a different path: the memory is automatically returned once the  
variable that owns it goes out of scope. Here’s a version of our scope example  
from Listing 4-1 using a String instead of a string literal:

In [28]:
{
    let s = String::from("Hello");  // s comes into scope
    // ... do stuff with s ...
}                                   // scope is now over, s is no longer valid
                                    // and rust auto calls drop() to free memory

There is a natural point at which we can return the memory our String needs to  
the allocator: when s goes out of scope. When a variable goes out of scope,  
Rust calls a special function for us. This function is called drop, and it’s  
where the author of String can put the code to return the memory. Rust calls  
drop automatically at the closing curly bracket.

Note: In C++, this pattern of deallocating resources at the end of an item’s  
lifetime is sometimes called Resource Acquisition Is Initialization (RAII).  
The drop function in Rust will be familiar to you if you’ve used RAII patterns.

This pattern has a profound impact on the way Rust code is written. It may seem  
simple right now, but the behavior of code can be unexpected in more complicated  
situations when we want to have multiple variables use the data we’ve allocated  
on the heap. Let’s explore some of those situations now.

### Variables and Data Interacting with Move
***

Multiple variables can interact with the same data in different ways in Rust.  
Let’s look at an example using an integer in Listing 4-2.

In [29]:
{
    let x = 5;
    let y = x; // assigning integer value of x to y
}

()

We can probably guess what this is doing: 
- “bind the value 5 to x; 
- then make a copy of the value in x
-  and bind it to y.” 

We now have two variables, x and y, and both equal 5. This is indeed what is  
happening, because integers are simple values with a known, fixed size, and  
these two 5 values are pushed onto the stack.

Now let’s look at the String version:

In [30]:
{
    let s1 = String::from("Hello");
    let s2 = s1; // s1 is moved to s2
}

()

This looks very similar, so we might assume that the way it works would be the  
same: 
- that is, the second line would make a copy of the value in s1 and bind it to s2.  

But this isn’t quite what happens...


<img src='https://doc.rust-lang.org/book/img/trpl04-01.svg' width=50% height=50%/>


> This is what is happening to String under the covers. A String is made up of  
three parts, shown on the left:  
- a pointer to the memory that holds the contents of the string, 
- a length, 
- and a capacity. 

> This group of data is stored on the stack. On the right is the memory on the  
heap that holds the contents.  

- The length is how much memory, in bytes, the contents of the String are  
currently using. 
- The capacity is the total amount of memory, in bytes, that the String has  
received from the allocator. 
- The difference between length and capacity matters, but not in this context,  
so for now, it’s fine to ignore the capacity.


When we assign s1 to s2, the String data is copied, meaning we :

- copy the pointer,   
- the length,  
- and the capacity that are on the stack. 

We do **`NOT`** copy the data on the **`HEAP`** that the pointer refers to.  
In other words, the data representation in memory looks like :

<img src="https://doc.rust-lang.org/book/img/trpl04-02.svg" width=50% height=50% />

Rust automatically calls the drop function and cleans up the heap memory for  
that variable and if memory were to work like the ABOVE ^ diagram ... with:  
- Both data pointers pointing to the same location...  
- when s2 and s1 go out of scope, they will both try to free the same memory!  
- This is known as a double free error ...  
Freeing memory twice can lead to memory corruption, which can potentially lead  
to security vulnerabilities.


**`THIS IS A PROBLEM`**

**`AND THIS IS HOW RUST FIXES IT!`**

### MOVE (GET OUT THE WAY LOL) 

> Because Rust also **`invalidates`** the first variable, instead of being called a  
shallow copy, it’s known as a **`move`**. In this example, we would say that s1 was  
moved into s2. So, what actually happens is shown :

<img src="https://doc.rust-lang.org/book/img/trpl04-04.svg" width=50% height=50%>

(In other languages this would be a shallow copy, because the first variable is  
NOT invalidated)

In [20]:
{
    let s1 = String::from("Hello"); // s1 comes into scope
    let s2 = s1;                    // pointer is moved to s2 ANDd invalidated at s1
                                    // s1 is NO LONGER VALID
                                    // So when we try to print s1 ...
    println!("{}, world!", s1);     // error: value borrowed here after move
}

Error: unused variable: `s2`

Error: borrow of moved value: `s1`

### Variables and Data Interacting with Clone

**`RUST DEFINITELY DOES NOT COPY THE HEAP BY DEFAULT => "CLONE" MUST BE CALLED**`


<img src="https://doc.rust-lang.org/book/img/trpl04-03.svg" width=50% height=50%>


**`RUST DEFINITELY DOES NOT COPY THE HEAP BY DEFAULT => "CLONE" MUST BE CALLED`**

If we do want to deeply copy the heap data of the String, not just the stack  
data, we can use a common method called clone. We’ll discuss method syntax in  
Chapter 5, but because methods are a common feature in many programming  
languages, you’ve probably seen them before.

In [32]:
{
    let s1 = String::from("Hello");
    let s2 = s1.clone(); // deep copy

    println!("s1 = {}, s2 = {}", s1, s2);
    println!("This works because we DEEP CLONED ... but we also made the code MORE EXPENSIVE");
}

s1 = Hello, s2 = Hello
This works because we DEEP CLONED ... but we also made the code MORE EXPENSIVE


()

This works just fine and explicitly **`DEEP CLONED the ENTIRE HEAP DATA`**

>When you see a call to clone, you know that some arbitrary code is being  
executed and that code may be **`EXPENSIVE`**.  

It’s a visual indicator that something different is going on.

### Stack Only : Data Copy

In [33]:
{
    let x = 5;
    let y = x;

    println!("x = {}, y = {}", x, y);   // According to the HEAP rules this should NOT work
                                        // But it does because integers are stored on the STACK
}

x = 5, y = 5


()

> But this code seems to contradict what we just learned: we don’t have a call to  
clone, but x is still valid and wasn’t moved into y.

The reason is that types such as integers that have a known size at compile  
time are stored entirely on the stack, so copies of the actual values are quick  
to make. That means there’s no reason we would want to prevent x from being  
valid after we create the variable y. In other words, there’s no difference  
between deep and shallow copying here, so calling clone wouldn’t do anything  
different from the usual shallow copying, and we can leave it out.  

### Copy Trait

> Rust has a special annotation called the Copy trait that we can place on types  
that are stored on the stack, as integers are  
(we’ll talk more about traits in Chapter 10).  

If a type implements the Copy trait, variables that use it do not move, but  
rather are trivially copied, making them still valid after assignment to another  
variable.

> Rust won’t let us annotate a type with **`Copy`** if the type, or any of its parts,  
has implemented the **`Drop`** trait. If the type needs something special to happen  
when the value goes out of scope and we add the Copy annotation to that type,  
we’ll get a compile-time error. To learn about how to add the Copy annotation  
to your type to implement the trait, see “Derivable Traits” in Appendix C.  

So, what types implement the Copy trait? You can check the documentation for the  
given type to be sure, but as a general rule, any group of simple scalar values  
can implement Copy, and nothing that requires allocation or is some form of  
resource can implement Copy. Here are some of the types that implement Copy:

@note All the integer types, such as u32.
- The Boolean type, bool, with values true and false.
- All the floating-point types, such as f64.
- The character type, char.
- Tuples, if they only contain types that also implement Copy.  
  For example, (i32, i32) implements Copy, but (i32, String) does not.

### Ownership and Functions

The mechanics of passing a value to a function are similar to those when  
assigning a value to a variable. Passing a variable to a function will move or  
copy, just as assignment does. 

Here is an example with some annotations showing where variables go into and out  
of scope:

In [42]:
{
    
    let s = String::from("Hello");  // s comes into scope
    println!("is s still valid? {}", s);    // yes s is still in scope
    takes_ownership(s);             // @note s's value FULLY moves into the function ...
                                    // ... and so is NO LONBER VALID in this scope

    // println!("is s still valid? {}", s);    // error: value borrowed here after move
                                                // and is no longer in scope

    let x = 5;                      // x comes into scope
    makes_copy(x);                  // x would move into the function, but it's COPIED
                                    // because i32 has a Copy (trait), so it's value
                                    // remains at this scope and AVAILABLE for use
    println!("is x still valid? {}", x);    // yes it is still 5 because i32 has a Copy trait

}   // After this closure :
    // - x goes out of scope
    // - then s ... but because it's VALUE was already moved ... 
    // @note going out of scope is IDEMPOTENT

fn takes_ownership(some_string: String){ // some_string comes into scope
    println!("[fn] takes ownership of {}", some_string);
}   // @note Here some_string goes out of scope and 'drop' is called 
    // the backing memory is freed!

fn makes_copy(some_integer: i32){   // some_integer comes into scope
    println!("[fn] makes copy of {}", some_integer);
}   // Here some_integer goes out of scope, but because i32 has a Copy Trait
    // @note Nothing special happens, this closure is IDEMPOTENT


is s still valid? Hello
[fn] takes ownership of Hello
[fn] makes copy of 5
is x still valid? 5


### Return Value and Scope

Returning values can also transfer ownership. Here is an example of a function  
that returns some value, with step by step annotations

In [48]:
{
    let s1 = gives_ownership(); // fn gives_ownership moves it's return value into s1
    println!("[fn] gives ownership of to s1 => {}", s1);

    let s2 = String::from("Hold my Beer"); // s2 comes into scope

    let s3 = takes_and_gives_back(s2);  // s2 values goes into fn ...
                                        // ... fn then moves it's returned value to s3
    // println!("is s2 still holding the beer {}", s2);    // error: value borrowed here after move
                                                        // s2 is invalid, it's value was moved to 
                                                        // fn ...which then moved it to s3
    println!("[fn] takes from s2 and gives back to s3 => {}", s3);

}   // Here s3 goes out of scope and is dropped
    // s2 was moved so nothing happened 
    // s1 goes out of scope and is dropped

fn gives_ownership() -> String {    // gives_ownership will move it's return value into the calling function
    let some_string = String::from("This is Your Book Now");    // some_string comes into scope
    some_string // some_string is moved out to the calling function
}               // scope closes but nothing special happens

fn takes_and_gives_back(a_string : String) -> String {  // will take ownership of a String and return a String
                // a_string comes into scope
    a_string    // a_string is moved out to the calling function
}               // scope closes but nothing special happens

[fn] gives ownership of to s1 => This is Your Book Now


[fn] takes from s2 and gives back to s3 => Hold my Beer


The ownership of a variable follows the same pattern every time: 
- assigning a value to another variable moves it. 
- When a variable that includes data on the heap goes out of scope, the value  
will be cleaned up by drop unless ownership of the data has been moved to  
another variable.

> @audit-ok : Drop is ONLY called on data that includes heap?

```
Yes, the concept of ownership and the `Drop` trait in Rust primarily apply to  
data that uses the heap. When a variable with heap-allocated data goes out of  
scope, the `drop` function is called to clean up the resources, unless ownership  
has been transferred to another variable.

For data types that are stored entirely on the stack, Rust follows a last-in,  
first-out (LIFO) order for cleaning up resources. When a variable goes out of  
scope, its memory is automatically reclaimed, and there's no need for explicit  
ownership management or dropping.

However, it's important to note that even for stack-allocated data, you can use  
ownership and borrowing concepts to control how the data is accessed and shared  
across your program. This can help you write safer and more efficient code.
```

While this works, taking ownership and then returning ownership with every  
function is a bit tedious. 

> What if we want to let a function use a value but not take ownership?  
It’s quite annoying that anything we pass in also needs to be passed back if we  
want to use it again, in addition to any data resulting from the body of the  
function that we might want to return as well.

Rust does let us return multiple values using a tuple

In [50]:
{
    let s1 = String::from("35lb KettleBell"); // s1 comes into scope

    let (s2, len) = calculate_length(s1);   // s1 is moved into fn calculate_length
                                            // fn returns ownership of s1 to s2, as well as the length

    println!("The length of '{}' is {}.", s2, len); 

}   // Here s2 goes out of scope and is dropped
    // len goes out of scope and is dropped
    // s1 is already dropped because it was moved to calculate_length
    // so nothing special happens

// takes ownership of a String to calculate it's length
// then returns ownership of the length and in String as a TUPLE
fn calculate_length(s: String) -> (String, usize) { // s is moved into scope
    // calculate the length of s
    let length = s.len(); // let comes into scope to store that length
    (s, length) // s and length are moved out to the calling function as a tuple
}

The length of '35lb KettleBell' is 15.


> But this is **`too much ceremony`** and a lot of work for a concept that should be  
common. Luckily for us, Rust has a feature for using a value without  
transferring ownership, called references.

References and Borrowing
---
***  

> The issue with the tuple code above ^ is that we have to return the String to  
the calling function so we can still use the String after the call to  
calculate_length, because the String was moved into calculate_length.  

**`Instead, we can provide a reference to the String value.`**  

A reference is like a pointer in that it’s an address we can follow to access  
the data stored at that address; that data is owned by some other variable.  
Unlike a pointer, a reference is guaranteed to point to a valid value of a  
particular type for the life of that reference.

Here is how you would define and use a calculate_length function that has a  
reference to an object as a parameter instead of taking ownership of the value:

### Immutable Reference

In [53]:
{
    let s1 = String::from("Hotel Key"); // s1 comes into scope
    // si is borrowed by calculate_length as an immutable reference (&)
    let len = calculate_length(&s1);
    // calculate_length goes out of scope ... 
    // @audit-ok : does anything happen to s1?
    // ANSWER : Nothing happens to s1 because it was borrowed as an immutable ref
    // ... like any other var, it's up to the owner (this scope) to drop it 
    println!("The length of '{}' is {}", s1, len);

}   // closure ends
    // len goes out of scope and is dropped
    // s1 goes out of scope and the owner (this scope) drops it

fn calculate_length(s: &String)-> usize{ // s is an immutable ref (borrowed))
    s.len() // len comes into scope and is (moved) returned to the calling function
}   // closure ends
    // s goes out of scope, but since we do NOT own it, we do NOT drop it
    // @audit-ok what do we do instead?
    // ANSWER : At this scope, NOTHING : it's the OWNER's responsibility to drop it!

The length of 'Hotel Key' is 9


When we pass &s1 into calculate_length and, in its definition we take : 
- **`&String`** 
- rather than **`String`**  

The ampersands **`&`** represent references, and they allow you to **`refer`** to some  
value **`without taking ownership`** of it


<img src="https://doc.rust-lang.org/book/img/trpl04-05.svg" width=50% height=50%>

> A diagram of **`&String`** s pointing at **`String`** s1


> Note: The opposite of referencing by using & is dereferencing, which is  
accomplished with the dereference operator, *. 

We call the action of **`creating a reference`** == **`borrowing`**  
As in real life, if a person owns something, you can borrow it from them.  
When you’re done, you have to give it back. **`You don’t own it.`**

In [54]:
// So, what happens if we try to modify something we’re borrowing?
// Let's fiddlestick around and find out LOL

{
    let s1 = String::from("Let's fiddlestick around and ..."); // s1 comes into scope
    change(&s1);    // s1 is borrowed as an immutable ref
                    // ... and passed to fn change
                    // ... which tries to change it
                    // ... but it can't because it's an immutable ref
                    // ... so it throws an error
    println!("s1 was ONLY borrowed, it better have come back in the SAME CONDITION {}", s1);
}   // closure ends
    // this scope owns s1, so it is our responsibility to drop it

// Let's find out what happens when we try to MODIFY an immutable ref
fn change(some_string: &String){        // &some_string comes in as an IMMUTABLE BORROW
    some_string.push_str("Find Out");   // Let's find out what happens if we try to change it
}   // closure ends
    // &some_string was borrowed, we DO NOT drop it ...
    // that's the RESPONSIBILITY of the OWNER

/*
** ERROR **
error[E0596]: cannot borrow immutable borrowed content `*some_string` as mutable
    some_string.push_str("Find Out");   // Let's find out what happens if we try to change it
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ `some_string` is a `&` reference, so the data it refers to cannot be borrowed as mutable
cannot borrow `*some_string` as mutable, as it is behind a `&` reference
help: consider changing this to be a mutable reference

&mut String
 */

Error: cannot borrow `*some_string` as mutable, as it is behind a `&` reference

**`Just as variables are immutable by default, so are references.`**  
We’re not allowed to modify something we have a reference to.

### Mutable Reference


In [56]:
// Let's fix the error in cell 54

{
    let mut s = String::from("hello"); // s comes into scope
    println!("{}", s);

    change(&mut s);     // adding the mut keyword to s signals that it is a MUTABLE REF
                        // ... so we can let fn change BORROW and MUTATE s
    println!("{}", s);
}   // end of closure
    // we OWN s, so we are responsible for dropping it
    // s goes out of scope and is dropped

// Adding the mut keyword to &some_string makes it a MUTABLE REF
fn change(some_string: &mut String){    // now we can BORROW and MUTATE some_string
    some_string.push_str(", world");    // mutate the borrowed value
}   // end of closure
    // since we only BORROWED some_string, we DO NOT drop it
    // that is the responsiblity fo the OWNER 
    // ... which is the calling function in this case

hello


hello, world


In [5]:
{
    let mut s = String::from("hello");

    let r1 = &mut s;
    let r2 = &mut s;
    println!("{}", r1);
    println!("{}", r2);
}

Error: cannot borrow `s` as mutable more than once at a time

This error says that this code is invalid because we **`cannot borrow s as mutable`**  
**`more than once at a time`**. The first mutable borrow is in r1 and must last until  
it’s used in the println!, but between the creation of that mutable reference  
and its usage, we tried to create another mutable reference in r2 that borrows  
the same data as r1.

The restriction preventing multiple mutable references to the same data at the  
same time allows for mutation but in a very controlled fashion. 

It’s something that new Rustaceans struggle with because most languages let you  
mutate whenever you’d like. The benefit of having this restriction is that Rust  
can prevent data races at compile time. A data race is similar to a race  
condition and happens when these three behaviors occur:

- Two or more pointers access the same data at the same time.
- At least one of the pointers is being used to write to the data.
- There’s no mechanism being used to synchronize access to the data.

Data races cause undefined behavior and can be difficult to diagnose and fix  
when you’re trying to track them down at runtime; Rust prevents this problem by  
refusing to compile code with data races!

As always, we can use curly brackets to create a new scope, allowing for   
multiple mutable references, just not simultaneous ones:


In [2]:
{
    let mut s = String::from("hello"); // s comes into scope as a mutable

    {
        let r1 = &mut s;  // r1 comes into the inner scope as a mutable ref
        println!("borrowing [s] mutable in [inner] scope {}", r1);
    } // r1 goes out of scope and is dropped here, releasing the mutable ref.

    let r2 = &mut s;    // since there are no mutable refs to s in the outer scope
                        // r2 can come into scope as a mutable ref to s
    println!("borrowing [s] mutable in [outer] scope {}", r2);
}   // closure ends
    // r2 goes out of scope and is dropped, RELEASING the mutable borrow on s
    // s goies out of scope and is dropped, DEALLLOCATING it's memory


// In actual RUST env ^ this could be outer scope
// BUT in Jupyter all rust code is in static scope so we needed to wrap it in 
// another BLOCK

borrowing [s] mutable in [inner] scope hello


borrowing [s] mutable in [outer] scope hello


Rust enforces a similar rule for combining mutable and immutable references.  
This code results in an error:

In [4]:
{
    let mut s = String::from("hello"); // s comes into scope as a mutable

    let r1 = &s; // no problem, r1 is an immutable ref to s
    let r2 = &s; // no problem, r2 is an immutable ref to s
    // let r3 = &s; // no problem, if r3 is an immutable ref to s
    let r3 = &mut s;    // BIG PROBLEM : 
                        // r3 is a mutable ref to s when r1 and r2 are still in scope
                        // when we want a mutable ref, it MUST be EXCLUSIVE
                        // because r1 and r2 are in scope, r3 can NOT have an EXCLUSIVE ref to s
                        // Else we risk data race where writes and read orders are not guaranteed
                        // AND can cause undefined behavior

    // This line will cause a compilation error due to the multiple borrow of s
    // - @audit-ok : the issue is that we are printing r1, r2, and r3 in the SAME CODE SPAN
    // - in this CODE SPAN the 2 reads and 1 write refs orders are NOT guaranteed
    // and as a result can cause UNDEFINED BEHAVIOR
    println!("{}, {}, and {}", r1, r2, r3);
}   // end of closure
    // r3 goes out of scope and is dropped, releasing it's ref
    // r2 goes out of scope and is dropped, releasing it's ref
    // r1 goes out of scope and is dropped, releasing it's ref 
    // s goes oot of scope and is dropped, DEALLOCATING it's memory (heap)

Error: cannot borrow `s` as mutable because it is also borrowed as immutable

> This code snippet demonstrates Rust's ownership and borrowing rules.  
Specifically, it shows that you can't have a mutable reference to a value while  
there are still active immutable references to the same value.  

This is a safety feature of Rust that helps prevent data races and ensures safe  
concurrency.

In this snippet, the :  
- `let r3 = &mut s;` line will cause a compilation error,  
- since `r1` and `r2` are still in scope, holding immutable references to `s`.  

> To fix this issue, you can either use only immutable references or ensure that  
the mutable reference is exclusive by dropping or going out of scope the  
immutable references before creating the mutable one.

We `cannot have a mutable reference while we have an immutable one` to  
the same value.

Users of an `immutable reference don’t expect the value to suddenly change` out  
from under them! However, multiple immutable references are allowed because no  
one who is just reading the data has the ability to affect anyone else’s  
reading of the data.

Note that a reference’s scope starts from where it is introduced and continues  
through the last time that reference is used. For instance, this code will  
compile because the last usage of the immutable references, the println!, occurs  
before the mutable reference is introduced:

**`RUST's Reference Acrobatics`**

reference read and write flows are done at very granular level CODE SPAN

> Explain SCOPE and CODE SPAN like I am 5

**Scope**: Imagine you have a toy box in your room. You can play with the toys  
only when you're in your room, and you can't play with them outside the room.  
In this case, your room is like the "scope" for your toys. It's the area where  
you can use and access your toys.

**Code Span**: Now, let's say you want to play with your toys in a specific way,  
like building a tower with your blocks. The time you spend building the tower is  
like a "code span." It's a specific activity or action you're doing with your  
toys.

> So, the "scope" is like the room where you can access and use your toys, while  
the "code span" is like the individual activities or actions you do with your  
toys.

In [6]:
// Rust's reference acrobatics :
// - this code WILL compile because the last usage of the immutable reference 
// (the println! macro) occurs BEFORE the mutable reference is introduced
// - @audit-ok : this GUARANTEES that there is NO RISK of r1, r2 reading STALE data from r3 writes!
{
    let mut s = String::from("hello"); // s comes into scope as a mutable

    /* ---------- CODE SPAN - [START] ------------ */
    let r1 = &s; // no problem : r1 is an immutable ref to s
    let r2 = &s; // no problem : r2 is an immutable ref to s

    println!("{} and {}", r1, r2); // LAST (and only) usage of r1 and r2
    // variables r1 and r2 will not be used after this point (code span)
    // @audit-ok : so if s is MUTATED after this point, there is NO RISK of  
    // r1 and r2 using stale data
    /* ---------- CODE SPAN - [END] ------------ */

    /* ---------- CODE SPAN - [START] ------------ */
    let r3 = &mut s;    // no problem, because there is no overlapping code span
                        // EVEN THOUGH r3 does NOT have EXCLUSIVE mutable access  to s
                        // @audit-ok : in this SCOPE, it has EXCLUSIVE access to s in this CODE SPAN
    println!("{}", r3); // NO RISK of data race between r1/r2 and r3 because :
                        // - @audit-ok : print(r1, r2) and print(r3) are in DIFFERENT CODE SPANS
    /* ---------- CODE SPAN - [END] ------------ */
}   // closure ends
    // r3 goes out of scope and is dropped, releasing it's ref
    // r2 goes out of scope and is dropped, releasing it's ref
    // r1 goes out of scope and is dropped, releasing it's ref
    // s goes out of scope and is dropped, DEALLOCATING it's memory (heap)

hello and hello


hello


> This code snippet illustrates how Rust's borrow checker ensures safe concurrency  
by enforcing the borrowing rules at compile time. In this example, the code  
compiles and runs correctly because the mutable reference `r3` is created and  
used after the last usage of the immutable references `r1` and `r2`.
  
This guarantees that there is no risk of data races between the references.

The comments provided give a clear explanation of why the code is safe and how  
Rust's ownership and borrowing system helps to prevent data races and ensure  
safe concurrency.

The scopes of the immutable references r1 and r2 end after the println! where  
they are last used, which is before the mutable reference r3 is created.  
These scopes don’t overlap, so this code is allowed: the compiler can tell that  
the reference is no longer being used at a point before the end of the scope.

Even though borrowing errors may be frustrating at times, remember that it’s the  
Rust compiler pointing out a potential bug early (at compile time rather than at  
runtime) and showing you exactly where the problem is. Then you don’t have to  
track down why your data isn’t what you thought it was.

### Dangling References

> Explain Dangling Reference like I am 5

Imagine you have a favorite toy car that you always play with. One day, your mom  
tells you it's time to clean up your room and put all your toys away. So, you  
put your toy car in a special box and your mom puts the box on a high shelf,  
where **`you can't reach it`**.

A "dangling reference" is like when you still remember where your toy car is and  
you try to play with it, but you can't because it's in the box on the high shelf.  
Just like you can't reach your toy car, a dangling reference in a program points  
to a memory location that it's not supposed to access anymore because the data  
has been cleaned up, just like your toy car was put away. This can cause  
problems and errors in the program, just like you might get sad or upset if you  
can't play with your favorite toy car.

***

In languages with pointers, it’s easy to erroneously create a dangling pointer  
— a pointer that references a location in memory that may have been given to  
someone else—by freeing some memory while preserving a pointer to that memory.  

In Rust, by contrast, the compiler guarantees that references will never be  
dangling references: 
- if you have a reference to some data, the compiler will ensure that the data  
will not go out of scope before the reference to the data does.  

Let’s try to create a dangling reference to see how Rust prevents them with a  
compile-time error:

In [8]:
{   // entering scope
    let reference_to_nothing = dangle(); // reference_to_nothing comes into scope
}   // exiting scope

fn dangle() -> &String{ // entering dangle scope
    let s = String::from("Just Hanging Around Bro"); // s enters scope 
    &s  // reference to s is returned to calling function
}   // exiting dangle scope
    // s goes out of scope and is dropped, DEALLOCACTING it's memory (heap)
    // - the string "Just Hanging Around Bro" no longer exists ...
    //   but &s ref still points to that address in memory

// This snippet will cause a compilation error
// because the main scope has a dangling reference to an address in memory that  
// is UNDEFINED

/*
fn dangle() -> &String{ // entering dangle scope
               ^ expected named lifetime parameter
missing lifetime specifier
help: this function's return type contains a borrowed value, but there is no value for it to be borrowed from
help: consider using the `'static` lifetime

'static
 */

Error: missing lifetime specifier

> This code snippet demonstrates a common mistake in Rust: 
- **`returning a reference to a local variable that goes out of scope`**  

In this example, the `dangle` function returns a reference to the local variable  
 `s`, which is deallocated when the function exits its scope.  This results in a  
dangling reference (`reference_to_nothing`) that points to an invalid memory  
location.

Rust's ownership system and borrow checker prevent this mistake at compile time,  
so this code will not compile.

> To fix this issue, you can return the `String` directly, transferring ownership  
to the calling function:

In [12]:
{   // entering scope
    let owned_value = no_dangle();  // owned value enters scope
                                    // scope is responsible for dropping owned_value
    println!("Valid address found for => {}", owned_value);
}   // exiting scope
    // owned_value goes out of scope and is dropped, DEALLOCATING it's memory (heap)

fn no_dangle() -> String { // no_dangle will return a string to the calling function
    let s = String::from("This Wandering String"); // s enters scope
    s   // s FULL OWNERSHIP is moved to to the calling function, on scope exit
        // unlike dangle() which returns only a REFERENCE to s on scope exit
}   // no_dangle scope ends
    // s ownership is moved to be the responsibility of the calling function

Valid address found for => This Wandering String


This code snippet demonstrates the correct way to return a value from a function  
in Rust without creating a dangling reference. In this example, the `no_dangle`  
function returns the `String` `s` directly, transferring its ownership to the  
calling function. The calling function's scope is then responsible for dropping  
the `owned_value` and deallocating its memory when it goes out of scope.

##### **`The Rules of Reference`**

Let’s recap what we’ve discussed about references:

At any given time, you can have either : 

- one mutable reference 
- or any number of immutable references.  

> References must always be valid.

The Slice Type
---
***

Slices let you reference a contiguous sequence of elements in a collection  
rather than the whole collection. 

> A slice is a kind of reference, so it does not have ownership.  

Here’s a small programming problem: 
- write a function that takes a string of words separated by spaces  
- and returns the first word it finds in that string. 
- If the function doesn’t find a space in the string, the whole string must be  
one word, so the entire string should be returned.

> Let’s work through how we’d write the signature of this function without using  
slices, to understand the problem that slices will solve:


```rust
// The first_word function has a &String as a parameter ...
// We don’t want ownership, so this is fine

fn first_word(s: &String) -> ? // But what should we return?
```

We don’t really have a way to talk about **`part of a string`**. However, we could  
**`return the index of the end of the word`**, indicated by a space. 

> Let’s try that :

In [18]:
fn first_word(s: &String) -> usize{
    let bytes = s.as_bytes();
    for (i, &item) in bytes.iter().enumerate(){
        if item == b' '{
            return i;   // return index of the first space char found
                        // this correlates to the end of the first word
        }
    }

    s.len() // no space found, we assume this is a single word
            // so we return the length of the string
}

{
    let mut s = String::from("hello world");
    let index_first_word = first_word(&s);
    println!("index_first_word [{}]", index_first_word);

    // let hello = &s[0..5];
    // let world = &s[6..11];
    let hello = &s[0..index_first_word];
    let world = &s[(index_first_word+1)..s.len()];

    println!("first half [{}] second half [{}]", hello, world);
}

index_first_word [5]
first half [hello] second half [world]


()

Rather than a reference to the entire String, hello is : 

- a reference to a portion of the String, 
- specified in the extra [0..5] bit.  

We create slices using a range within brackets by specifying  
[starting_index..ending_index], where starting_index is the first position in  
the slice and ending_index is one more than the last position in the slice.  

Internally, the slice data structure stores :  
- the starting position and the length of the slice, 
- which corresponds to ending_index minus starting_index. 

> So, in the case of let world = &s[6..11];, world would be a slice that contains:   
> - a pointer to the byte at index 6 of s  
> - with a length value of 5.

<img src="https://doc.rust-lang.org/book/img/trpl04-06.svg" width=50% height=50%>

> Help explain rust owner ship with respect to shadowing variables : In a given closure, when I let var1 = 1, then I shadow it with var1  = 2, walk me through what is happening with ownership while in a closure and what happens when that closure exits

In [24]:
// Rust slice syntax explictt vs implicit examples
{
    let s = String::from("hello");  // s comes into scope
    let len = s.len();              // len comes into scope

    let slice = &s[3..len]; // slice comes into scope
    println!("slice [3..len] => {slice} // explicit start/end expression");
    let slice = &s[3..];    // slice
    println!("slice [3..]    => {slice} // '[start..]' implicit goes to end");

    let slice = &s[0..len];
    println!("slice [0..len] => {slice} // explict 0 to len of full slice");
    let slice = &s[..];
    println!("slice [..]     => {slice} // implicit .. begin to end full slice");
}

slice [3..len] => lo // explicit start/end expression
slice [3..]    => lo // '[start..]' implicit goes to end
slice [0..len] => hello // explict 0 to len of full slice
slice [..]     => hello // implicit .. begin to end full slice


()

> Note: String slice range indices must occur at valid UTF-8 character  
boundaries. If you attempt to create a string slice in the middle of a multibyte  
character, your program will exit with an error. For the purposes of introducing  
string slices, we are assuming ASCII only in this section; a more thorough  
discussion of UTF-8 handling is in the “Storing UTF-8 Encoded Text with Strings”  
section of Chapter 8.

A real "mature" Example lol

Shadow MIGHT be ok to track a ==VALUE== across ==TYPE casting==, like from  
&str to char #i-might-not-stab-you

In [44]:
{   // enter closure
    let s = String::from("Giving her a D"); // s comes into scope
    let len = s.len();  // len comes into scope

    // retrives the last character "D" as a string slice
    let d_for_her = &s[(len-1)..len]; // (a) d_for_her comes into scope

    // verify that type is &string
    let mut type_name = get_type_name(&d_for_her); // mut type_name comes into scope
    println!("her preference => {} len > {} of type [= {} =]", d_for_her, d_for_her.len(), type_name);

    // SHADOW (a) d_for_her => &str with (b) d_for_her => char
    let d_for_her = d_for_her.chars().next().unwrap(); // (b) d_for_her comes into scope
    // verify that type is now char
    type_name = get_type_name(&d_for_her); 
    println!("the size she's getting {} of type [= {} =] lol", d_for_her, type_name);
}   // Exit closure
    // Variables go out of scope and are dropped in reverse order:
    // - d_for_her (b - shadow of type charr) is dropped (char is popped off the STACK)
    // - type_name is dropped
    // - d_for_her (a - shadowed of type &str) is dropped, DEALLOCATING it's memory (heap)
    // - len is dropped
    // - s is dropped, DEALLOCATING it's memory (heap)
    // Closure exited

her preference => D len > 1 of type [= &str =]
the size she's getting D of type [= char =] lol


With all this information in mind, let’s rewrite first_word to return a slice.  
The type that signifies “string slice” is written as &str:

In [10]:
// Define a function that takes a reference to a string and returns a string slice
fn first_word(s: &str) -> &str{  // scope starts
                                    // s is a reference to a String, 
                                    // - no ownership is taken
                                    // - scope is NOT responsible for dropping s
                                    // scope will return a reference to a slice of s
                                    // to the calling function to TAKE OWNERSHIP of
    let bytes = s.as_bytes();   // bytes comes into scope (it's a &[u8])
                                // ... but because the length of the byte array 
                                // is UNKNOWN at compile time => HEAP

    for (i, item) in bytes.iter().enumerate(){
        if *item == b' '{
            // on first item to == ' ' return the slice to this index
            // - this is our first word
            return &s[0..i] // scope will return a reference to a slice of s
        }
    }

    // else return the full slice
    // - either it's one word
    // - or no space was found
    // - or no words???
    &s[..] // scope will return a reference to a slice of s
}   // scope ends
    // bytes is dropped, DEALLOCATING it's memory (heap)
    // - @note because the size of the byte array is UNKNOWN at compile time => HEAP

{   // enter closure
    let mut s = String::from("terrible lunch walk conversation with Amy"); // s comes into scope

    let word = first_word(&s);  // word comes into scope
    // s.clear(); // error! @audit ... I don't understand what the issue is
    println!("first word => {}", word);
}   // exit closure
    // WRONG!!** word is dropped, DEALLOCATING it's memory (heap) ** WRONG!!!
    // word is a REFERENCE to a SLICE of s ... and are stored on the STACK
    // - word is dropped but NO HEAP memory is deallocated
    // s is dropped, DEALLOCATING it's memory (heap)


first word => terrible


()

Recall from the borrowing rules that if we have an immutable reference to  
something, we cannot also take a mutable reference. 

- Because `clear` needs to truncate the String, it `needs to get a mutable reference`. 
- The `println!` after the call to clear `uses the reference in word`, so the  
immutable reference `must still be active at that point`. 

Rust disallows : 
- the mutable reference in **`clear`** 
- AND the immutable reference in **`word`** from `existing at the same time` 
- compilation FAILS as a result

> Not only has Rust made our API easier to use, but it has also eliminated an  
entire class of errors at compile time!

We now have a straightforward API that’s much harder to mess up because the  
compiler will ensure the references into the String remain valid. Remember the  
bug in the program in Listing 4-8, when we got the index to the end of the first  
word but then cleared the string so our index was invalid? That code was  
logically incorrect but didn’t show any immediate errors. 

The problems would show up later if we kept trying to use the first word index  
with an emptied string.  

> Slices make this bug impossible and let us know we have a problem with our code  
much sooner.  

Using the slice version of first_word will throw a compile-time error:

### String Literals as Slices

Recall that we talked about string literals being stored inside the binary. Now  
that we know about slices, we can properly understand string literals:

```rust
let s = "Hello, world!";
```

The type of s here is &str: it’s a slice pointing to that specific point of the  
binary. This is also why string literals are immutable; &str is an immutable  
reference.

##### String Slices as Parameters

Knowing that you can take slices of literals and String values leads us to one  
more improvement on first_word, and that’s its signature:

```rust
fn first_word(s: &String) -> &str { ... }
```

A more experienced Rustacean would write the signature shown in Listing 4-9  
instead because it allows us to use the same function on both &String values  
and &str values.

```rust
fn first_word(s: &str) -> &str { ... }
```

> Listing 4-9: Improving the first_word function by using a string slice for the  
type of the s parameter

If we have a string slice, we can pass that directly. If we have a String, we  
can pass a slice of the String or a reference to the String. This flexibility  
takes advantage of deref coercions, a feature we will cover in  
“Implicit Deref Coercions with Functions and Methods” section of Chapter 15.

Defining a function to take a string slice instead of a reference to a String  
makes our API more general and useful without losing any functionality:

In [24]:
{
    let my_string = String::from("hello world");

    // first_word works on slices of 'String's, whether partial or full
    let word = first_word(&my_string[0..4]);
    println!("first word => &my_string[0..4] [{}]", word); // hell
    let word = first_word(&my_string[..]);
    println!("first word => &my_string[..] [{}]", word); // hello
    
    // first_word also works on reference to Strings, which are equivalent
    // to whole slices of Strings
    let word = first_word(&my_string);
    println!("first word => &my_string [{}]", word); // hello

    let my_string_literal = "heyao world";
    
    // first_word works on slicees of literals, whether partial or whole
    let word = first_word(&my_string_literal[0..3]);
    println!("first word => &my_string_literal[0..3] [{}]", word); // hey
    let word = first_word(&my_string_literal[..]);
    println!("first word => &my_string_literal[..] [{}]", word); // heyao

    // because string literals *are* string slices already,
    // this works too, without the slice syntax!
    let word = first_word(my_string_literal);
    println!("first word => &my_string_literal [{}]", word); // heyao

}

first word => &my_string[0..4] [hell]
first word => &my_string[..] [hello]
first word => &my_string [hello]
first word => &my_string_literal[0..3] [hey]
first word => &my_string_literal[..] [heyao]
first word => &my_string_literal [heyao]


()

##### Other Slices

String slices, as you might imagine, are specific to strings. But there’s a more  
general slice type too. Consider this array:

```rust
let a = [1, 2, 3, 4, 5];
```
Just as we might want to refer to part of a string, we might want to refer to  
part of an array. We’d do so like this:

```rust
let a = [1, 2, 3, 4, 5];

let slice = &a[1..3];

assert_eq!(slice, &[2, 3]);
```
This slice has the type &[i32]. It works the same way as string slices do, by  
storing a reference to the first element and a length. You’ll use this kind of  
slice for all sorts of other collections. We’ll discuss these collections in  
detail when we talk about vectors in Chapter 8.

SUMMARY
---

The concepts of ownership, borrowing, and slices ensure memory safety in Rust  
programs at compile time. The Rust language gives you control over your memory  
usage in the same way as other systems programming languages, but having the  
owner of data automatically clean up that data when the owner goes out of scope  
means you don’t have to write and debug extra code to get this control.

Ownership affects how lots of other parts of Rust work, so we’ll talk about  
these concepts further throughout the rest of the book. Let’s move on to  
Chapter 5 and look at grouping pieces of data together in a struct.